## Load packages

In [14]:
import rdflib, glob
from rdflib import URIRef, Literal
from rdflib.namespace import RDF, RDFS, DCTERMS, SKOS

## Define namespaces

In [2]:
abrc = rdflib.Namespace("https://w3id.org/anything-but-routine/4.0/classification/")
abri = rdflib.Namespace("https://w3id.org/anything-but-routine/4.0/instance/")
abrw = rdflib.Namespace("https://w3id.org/anything-but-routine/4.0/work/")
bf = rdflib.Namespace("http://id.loc.gov/ontologies/bibframe/")
arm = rdflib.Namespace("https://w3id.org/arm/core/ontology/0.1/")

In [3]:
def initialize_abr_graph():
    g = rdflib.Graph()
    g.bind("abrc", "https://w3id.org/anything-but-routine/4.0/classification/")
    g.bind("abri", "https://w3id.org/anything-but-routine/4.0/instance/")
    g.bind("abrw", "https://w3id.org/anything-but-routine/4.0/work/")
    g.bind("bf", "http://id.loc.gov/ontologies/bibframe/")
    g.bind("arm", "https://w3id.org/arm/core/ontology/0.1/")
    return g

## Add bf:Classifications for ABR v4.0

In [15]:
scheme = 'Schottlaender_v4_0'

categories = {
    'A': 'BOOKS, BROADSIDES, AND PAMPHLETS',
    'B': 'ART BOOKS, EXHIBITION CATALOGUES, AND RELATED ITEMS',
    'C': 'CONTRIBUTIONS TO PERIODICALS',
    'D': 'FOREIGN TRANSLATIONS',
    'E': 'SOUND RECORDINGS',
    'F': 'VIDEO RECORDINGS',
    'G': 'MISCELLANEOUS',
    'H': 'BIOGRAPHY, INTERVIEWS, AND LETTERS',
    'I': 'BIBLIOGRAPHY AND CRITICISM'
}

category_definitions = {
    'A': '"... Burroughs’ publications in book, broadside, and pamphlet form. Foreign titles are included only if they have no English-language analogue (e.g., A16) or represent the first publication of a particular title (e.g., A34a)."-Introduction.',
    'B': '"... Burroughs’ art books (several of which were co-authored) and catalogs of exhibitions of his art."-Introduction.',
    'C': '"... Burroughs’ many hundreds of contributions to periodicals."-Introduction.',
    'D': '"... foreign translations of Burroughs’ work."-Introduction.',
    'E': '"... Burroughs’ own sound recordings, and his contributions to or appearances on those by others."-Introduction.',
    'F': '"... the video formats in which Burroughs’ early film work has been made available, along with various of Burroughs’ readings and other Burroughs-related material that have also been made available as video recordings."-Introduction.',
    'G': '"... various miscellaneous items which, while most are not by Burroughs, are nevertheless relevant in and to his bibliography."-Introduction.',
    'H': '"... biographies of, interviews with, and letters by Burroughs"-Introduction.',
    'I': '"... bibliographies and criticism of him [Burroughs]. Bibliographic and critical works are included only if issued as stand-alone publications, and only if they deal exclusively with Burroughs or if he is named in the title."-Introduction.'    
}

h = initialize_abr_graph()
scheme_uri = abrc[scheme]
h.add((uri, RDF.type, bf.Classification))
h.add((uri, RDFS.label, Literal('Schottlaender v4.0')))
h.add((uri, SKOS.definition, Literal('The broad categorization scheme used in Brian E.C. Schottlaender’s "ANYTHING BUT ROUTINE: A Selectively Annotated Bibliography of William S. Burroughs v. 4.0"')))
h.serialize(f"edited-ttl/classification/{scheme}.ttl", format='turtle')

for category, label in categories.items():
    uri = abrc[category]
    h = initialize_abr_graph()
    h.add((uri, RDF.type, bf.Classification))
    h.add((uri, RDFS.subClassOf, scheme_uri))
    h.add((uri, RDFS.label, Literal(label)))
    h.add((uri, SKOS.definition, Literal(category_definitions[category])))
    h.serialize(f"edited-ttl/classification/{category}.ttl", format='turtle')

## Add bf:classification to bf:Works, bf:Instances

In [16]:
for infile in glob.glob("edited-ttl/instance/*.ttl"):
    g = initialize_abr_graph()
    g.parse(infile, format='n3')
    for instance in g.subjects(RDF.type, bf.Instance):
        g.add((instance, bf.classification, abrc['A']))
    g.serialize(infile, format='turtle')

In [17]:
for infile in glob.glob("edited-ttl/work/*.ttl"):
    g = initialize_abr_graph()
    g.parse(infile, format='n3')
    for work in g.subjects(RDF.type, bf.Work):
        g.add((work, bf.classification, abrc['A']))
    g.serialize(infile, format='turtle')